In [1]:
# ====================== Run Flower server & clients ======================
import subprocess, threading, time, socket, sys, os, signal, shlex
from pathlib import Path
from collections import deque
from typing import Optional, List

PY = sys.executable or "python3"

# stato globale
server_process: Optional[subprocess.Popen] = None
client_processes: list[subprocess.Popen] = []
client_threads: list[threading.Thread] = []
server_log_buffer = deque(maxlen=2000)  # buffer log server


# -------------------- helpers --------------------
def _stream_output(process: subprocess.Popen, tag: str, buffer: deque | None = None):
    """Legge stdout del processo e stampa riga per riga in tempo reale (e salva nel buffer, se passato)."""
    try:
        assert process.stdout is not None
        for line in iter(process.stdout.readline, ''):
            if not line:
                break
            line = line.rstrip()
            if buffer is not None:
                buffer.append(line)
            print(f"[{tag}] {line}")
    except Exception as e:
        print(f"[{tag}] <stream error: {e!r}>")
    finally:
        rc = process.wait()
        print(f"[{tag}] terminato con codice {rc}")


def _wait_port(host: str, port: int, timeout: float = 30.0) -> bool:
    t0 = time.time()
    while time.time() - t0 < timeout:
        try:
            with socket.socket() as s:
                s.settimeout(1.0)
                s.connect((host, port))
                return True
        except OSError:
            time.sleep(0.25)
    return False


def _is_port_free(host: str, port: int, timeout: float = 0.5) -> bool:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(timeout)
    try:
        s.connect((host, port))
        s.close()
        return False  # connessione riuscita → porta occupata
    except Exception:
        return True   # non raggiungibile → porta libera


# -------------------- server --------------------
def start_server(
    address: str = "127.0.0.1:8081",
    *,
    server_file: str = "server.py",
    eval_csv: str = "data/public_eval.csv",
    rounds: int = 5,
    trees_per_round: int = 30,
    pool_max_trees: int = 1000,
    patience: int = 3,
    # opzionali (se None non vengono passati)
    fraction_fit: float | None = None,
    min_fit_clients: int | None = None,
    min_available_clients: int | None = None,
    # processo
    cwd: str | None = None,
    python_bin: str | None = None,
    check_port: bool = True,
    extra_args: List[str] | None = None,
):
    global server_process, server_log_buffer
    server_log_buffer.clear()

    base_cwd = cwd or os.getcwd()
    sf = Path(server_file if cwd is None else os.path.join(base_cwd, server_file))
    if not sf.exists():
        print(f"'{server_file}' non trovato (cwd={base_cwd})")
        return None

    if server_process and server_process.poll() is None:
        print("Server già attivo.")
        return server_process

    # parsing host:port per healthcheck
    try:
        host, port_str = address.split(":")
        port = int(port_str)
    except Exception:
        print(f"Indirizzo non valido: '{address}' (usa HOST:PORT)")
        return None

    if check_port and not _is_port_free(host, port):
        print(f"La porta {address} è occupata. Chiudi il processo che la usa oppure cambia porta.")
        return None

    py = python_bin or PY
    cmd = [
        py, str(sf),
        "--server", address,
        "--eval_csv", eval_csv,
        "--rounds", str(rounds),
        "--trees_per_round", str(trees_per_round),
        "--pool_max_trees", str(pool_max_trees),
        "--patience", str(patience),
    ]
    # opzionali se forniti
    if fraction_fit is not None:
        cmd += ["--fraction_fit", str(fraction_fit)]
    if min_fit_clients is not None:
        cmd += ["--min_fit_clients", str(min_fit_clients)]
    if min_available_clients is not None:
        cmd += ["--min_available_clients", str(min_available_clients)]
    if extra_args:
        cmd += list(extra_args)

    print("Avvio server →", " ".join(shlex.quote(c) for c in cmd))
    server_process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        cwd=base_cwd,
    )
    threading.Thread(
        target=_stream_output,
        args=(server_process, "SERVER", server_log_buffer),
        daemon=True,
    ).start()

    ok = _wait_port(host, port, timeout=30.0)
    if ok:
        print(
            f"Server in ascolto su {address} "
            f"(rounds={rounds}, trees_per_round={trees_per_round}, pool_max_trees={pool_max_trees})"
        )
    else:
        print("La porta non è raggiungibile: il server potrebbe essere crashato.")
        print("— Ultimi log del server —")
        for ln in list(server_log_buffer)[-40:]:
            print("[SERVER][last]", ln)
    return server_process



# -------------------- clients --------------------
def start_clients(
    n: int,
    server_address: str = "127.0.0.1:8081",
    *,
    # mapping client index: usa 0-based o 1-based
    start_index: int = 1,                # 1 → client1..clientN ; 0 → client0..client{n-1}
    num_clients_flag: int | None = None, # se non None → passa --num_clients al client

    # Opzioni processo
    client_file: str = "client.py",
    cwd: str | None = None,
    python_bin: str | None = None,
    stagger_sec: float = 0.3,

    # Argomenti extra pass-through alla CLI del client (per TUTTI i client)
    extra_args: list[str] | None = None,

    # ---- Config attacchi (solo per client maliziosi) ----
    malicious_ids: list[str] | None = None,         
    attack_mode_for_malicious: str = "backdoor",     # "backdoor" | "random_model" | "label_flip"
    # backdoor
    poison_frac: float = 0.15,
    trigger_delta: float = 8.0,
    trigger_cols: list[int] | None = None,           
    target_class: int = 1,
    # random_model
    random_model_depth: int = 6,


    per_client_extra: dict[str, list[str]] | None = None,
):
    """
    Avvia n client: indici start_index .. start_index+n-1
    Richiede che client.py supporti: --server --client_id
    - I client presenti in 'malicious_ids' ricevono anche --is_malicious e i parametri dell'attacco.
    - 'per_client_extra' permette override fini-grana per singolo client (si somma a quanto sopra).
    """
    import os, shlex, subprocess, threading, time
    from pathlib import Path

    # Queste global devon esistere nel modulo chiamante
    global client_processes, client_threads, PY, _stream_output

    base_cwd = cwd or os.getcwd()
    cf = Path(client_file if cwd is None else os.path.join(base_cwd, client_file))
    if not cf.exists():
        print(f"'{client_file}' non trovato (cwd={base_cwd})")
        return []

    py = python_bin or PY
    launched = []

    malicious_ids = set(malicious_ids or [])
    per_client_extra = per_client_extra or {}

    # Prepara stringa trigger_cols se fornita
    trigger_cols_str = None
    if trigger_cols is not None:
        trigger_cols_str = ",".join(str(i) for i in trigger_cols)

    for i in range(start_index, start_index + n):  # <-- corretto off-by-one
        client_id = f"client{i}"
        cmd = [
            py, str(cf),
            "--server", server_address,
            "--client_id", client_id,
        ]

        if num_clients_flag is not None:
            cmd += ["--num_clients", str(num_clients_flag)]

        # Argomenti comuni a tutti
        if extra_args:
            cmd += list(extra_args)

        # Se client malizioso: aggiungi attack flags standard
        if client_id in malicious_ids:
            cmd += ["--is_malicious", "--attack_mode", attack_mode_for_malicious]
            if attack_mode_for_malicious == "backdoor":
                cmd += ["--poison_frac", str(poison_frac),
                        "--trigger_delta", str(trigger_delta)]
                if trigger_cols_str is not None:
                    cmd += ["--trigger_cols", trigger_cols_str]
                cmd += ["--target_class", str(target_class)]
            elif attack_mode_for_malicious == "random_model":
                cmd += ["--random_model_depth", str(random_model_depth)]
            elif attack_mode_for_malicious == "label_flip":
                # nessun flag obbligatorio aggiuntivo, mappa default interna
                pass

        # Override/aggiunte specifiche per client (si applicano alla fine)
        if client_id in per_client_extra:
            cmd += list(per_client_extra[client_id])

        print("Avvio", client_id, "→", " ".join(shlex.quote(c) for c in cmd))
        p = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            cwd=base_cwd,
        )
        client_processes.append(p)
        th = threading.Thread(target=_stream_output, args=(p, f"CLIENT {i}"), daemon=True)
        th.start()
        client_threads.append(th)
        launched.append(i)
        time.sleep(stagger_sec)

    return launched



In [2]:
start_server(
    address="127.0.0.1:8082",
    server_file="server_model.py",
    eval_csv="data/public_eval.csv",
    rounds=10,
    trees_per_round=30,
    pool_max_trees=10000,
    patience=3,
    # opzionali:
    # fraction_fit=0.6,
    min_fit_clients=6,
    min_available_clients=6,
    check_port=True,
)

Avvio server → /home/habes/anaconda3/envs/marcoenv96/bin/python server_model.py --server 127.0.0.1:8082 --eval_csv data/public_eval.csv --rounds 10 --trees_per_round 30 --pool_max_trees 10000 --patience 3 --min_fit_clients 6 --min_available_clients 6
[SERVER] 
[SERVER] WARNING :   DEPRECATED FEATURE: flwr.server.start_server() is deprecated.
[SERVER] 	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:
[SERVER] 
[SERVER] 		$ flower-superlink --insecure
[SERVER] 
[SERVER] 	To view usage and all available options, run:
[SERVER] 
[SERVER] 		$ flower-superlink --help
[SERVER] 
[SERVER] 	Using `start_server()` is deprecated.
[SERVER] 
[SERVER]             This is a deprecated feature. It will be removed
[SERVER]             entirely in future versions of Flower.
[SERVER] 
[SERVER] INFO :      Starting Flower server, config: num_rounds=10, no round_timeout
[SERVER] INFO :      Flower ECE: gRPC server running (10 rounds), SSL is disabled
[SERVER] INFO :      [

<Popen: returncode: None args: ['/home/habes/anaconda3/envs/marcoenv96/bin/p...>

[SERVER] INFO :      Received initial parameters from one random client
[SERVER] INFO :      Starting evaluation of initial global parameters
[SERVER] INFO :      initial parameters (loss, other metrics): 0.0, {'note': 'no_trees_yet'}
[SERVER] INFO :
[SERVER] INFO :      [ROUND 1]
[SERVER] INFO :      configure_fit: strategy sampled 6 clients (out of 6)


In [3]:
start_clients(
    n=6,
    server_address="127.0.0.1:8082",
    start_index=1,
    malicious_ids=["client3", "client4"],
    #malicious_ids=[""],
    attack_mode_for_malicious="sign_flip",
    poison_frac=1,#0.50,  
    #trigger_delta=8,
)


Avvio client1 → /home/habes/anaconda3/envs/marcoenv96/bin/python client.py --server 127.0.0.1:8082 --client_id client1
Avvio client2 → /home/habes/anaconda3/envs/marcoenv96/bin/python client.py --server 127.0.0.1:8082 --client_id client2
Avvio client3 → /home/habes/anaconda3/envs/marcoenv96/bin/python client.py --server 127.0.0.1:8082 --client_id client3 --is_malicious --attack_mode sign_flip
Avvio client4 → /home/habes/anaconda3/envs/marcoenv96/bin/python client.py --server 127.0.0.1:8082 --client_id client4 --is_malicious --attack_mode sign_flip
[CLIENT 1] WARNING :   DEPRECATED FEATURE: flwr.client.start_numpy_client() is deprecated.
[CLIENT 1] 	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below:
[CLIENT 1] 	flwr.client.start_client(
[CLIENT 1] 		server_address='<IP>:<PORT>',
[CLIENT 1] 		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
[CLIENT 1] 	)
[CLIENT 1] 	Using `sta

[1, 2, 3, 4, 5, 6]

[CLIENT 4] WARNING :   DEPRECATED FEATURE: flwr.client.start_numpy_client() is deprecated.
[CLIENT 4] 	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below:
[CLIENT 4] 	flwr.client.start_client(
[CLIENT 4] 		server_address='<IP>:<PORT>',
[CLIENT 4] 		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
[CLIENT 4] 	)
[CLIENT 4] 	Using `start_numpy_client()` is deprecated.
[CLIENT 4] 
[CLIENT 4]             This is a deprecated feature. It will be removed
[CLIENT 4]             entirely in future versions of Flower.
[CLIENT 4] 
[CLIENT 4] WARNING :   DEPRECATED FEATURE: flwr.client.start_client() is deprecated.
[CLIENT 4] 	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:
[CLIENT 4] 
[CLIENT 4] 		$ flower-supernode --insecure --superlink='<IP>:<PORT>'
[CLIENT 4] 
[CLIENT 4] 	To view all available options, run:
[CLIENT 4] 
[CLIENT 4] 		$ flower-super

In [ ]:
start_clients(
    n=6,
    server_address="127.0.0.1:8081",
    start_index=1,
    malicious_ids=["client3"],
    attack_mode_for_malicious="backdoor",
    poison_frac=0.25,              # 25% campioni avvelenati
    trigger_delta=10.0,            # offset numerico sulle feature trigger
    trigger_cols=[0, 1, 2],        # colonne trigger
    target_class=1,                # classe obiettivo
)


In [ ]:
start_clients(
    n=6,
    server_address="127.0.0.1:8081",
    start_index=1,
    malicious_ids=["client3"],
    attack_mode_for_malicious="backdoor",
    poison_frac=0.25,              # 25% campioni avvelenati
    trigger_delta=10.0,            # offset numerico sulle feature trigger
    trigger_cols=[0, 1, 2],        # colonne trigger
    target_class=0,                # classe obiettivo
)
